In [ ]:
from openai import OpenAI
import pandas as pd
import ast
import jsonlines
import os

In [ ]:
client = OpenAI()
MODEL='gpt-4o'

# Aggrefact prompt
system = """Split the input paragraph into a list of sentences, each accompanied by its start and end index in the original paragraph. Please make sure that your output is a list of tuples in the format: (sentence, start_index, end_index). Each extracted sentence must remain exactly as it appears in the input, preserving all characters, spaces, and formatting without any modifications. Note that spaces at the end of a sentence should be considered part of that sentence. The space at the begining of input should be considered part of the first sentence.

For example:
Input: ' Here is a concise summary of the passage:  * More than cases of COVID-19 have been reported * Cases have been reported in more than 190 countries and 200 territories * More than deaths have resulted from COVID-19'
Output: [(" Here is a concise summary of the passage:  ", 0, 44), ("* More than cases of COVID-19 have been reported ", 44, 93), ("* Cases have been reported in more than 190 countries and 200 territories ", 93, 167), ("* More than deaths have resulted from COVID-19", 167, 213)]
"""
user = 'Make sure that your output is a string of a list of tuples in the format: (sentence, start_index, end_index). It should be able to be compiled by ast.literal_eval\n\nInput: {summary}\nOutput:'

def call_gpt(system_prompt, user_prompt, model='gpt-4', temperature=0):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )

    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [ ]:
df = pd.read_csv('../../assign/examples_to_annotate.csv')

existing_meta_ids = []
if os.path.exists('summary_sent_list.jsonl'):
    with open('summary_sent_list.jsonl') as r:
        for p in jsonlines.Reader(r):
            existing_meta_ids.append(p[0]['meta_id'])
print(existing_meta_ids)

for index, row in df.iterrows():
    print(int(index))
    if int(index) in existing_meta_ids:
        continue
    
    result = call_gpt(system, user.format(summary=row['summary']), model=MODEL).replace('```python','').replace('```','')
    try:
        sent_list = ast.literal_eval(result)
        for idx, sent_record in enumerate(sent_list):
            sent, start_idx, end_idx = sent_record
            if len(sent) > 350:
                print(len(sent))
                raise ValueError(f"The {idx}-th sentence in sample {int(index)} is too long. Need double check.")
            start_index = row['summary'].find(sent)
            if start_index != -1:
                # Calculate the end index
                end_index = start_index + len(sent) - 1
                print(f"Start index: {start_index}, End index: {end_index}")
                if start_idx != start_index:
                    sent_list[idx][1] = start_index
                if end_idx != end_index:
                    sent_list[idx][2] = end_index
            else:
                try:
                    sent = sent.replace('. ','.\n\n')
                    start_index = row['summary'].find(sent)
                    if start_index != -1:
                        # Calculate the end index
                        end_index = start_index + len(sent) - 1
                        print(f"Start index: {start_index}, End index: {end_index}")
                        sent_list[idx][0] = sent
                        if start_idx != start_index:
                            sent_list[idx][1] = start_index
                        if end_idx != end_index:
                            sent_list[idx][2] = end_index
                        
                except:
                    raise ValueError(f"Meta_id: {int(index)}. The {idx}-th sentence {sent} not found in summary.")
        
        item = [{'meta_id':int(index), 'summary': row['summary'], 'sent_list': result}]
        print(item)
        if os.path.exists('summary_sent_list.jsonl'):
            mode = 'a'
        else:
            mode = 'w'
        with jsonlines.open('summary_sent_list.jsonl', mode=mode) as writer:
            writer.write(item)
    except:
        print(result)
        break
 